In [90]:
## Import the necessary libraries

import numpy as np
import cv2
import face_recognition
import os
from datetime import datetime
import time
import pandas as pd
import os.path
from os import path

## We need the path of each of the images, and their names

path = 'Recognition'   # make any folder in which images are present
images = []            # empty list to get the location of the images
classNames = []        # Empty list to get out the name from the images.

## Grab the list of the images in the directory

myList = os.listdir(path)


# Now read the images and take out the names from myList

for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])
     

# Start the encoding process of the images

def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

#def markAttendance(name):
   # with open('Attendance.csv','r+') as f:   # Create a csv file named Attendance with Name and time heading.
      ##  myDataList = f.readlines()    
       ## nameList = []
       ## for line in myDataList:        # read throug each line 
        #    entry = line.split(',')
        #    nameList.append(entry[0])    # appends the name in the first column
            
       # if name not in nameList:         # If name not present already in the list then puts the time 
        #    now = datetime.now()
        ##    dtString = now.strftime('%H%M%S')
         #   f.writelines(f'\n{name},{dtString}')

def create_folder():
    today = datetime.now()
    
    attendance_date = today.strftime('%Y%m%d')
    if (os.path.exists("Attendance/" + today.strftime('%Y%m%d'))== False):
        os.mkdir("Attendance/" + today.strftime('%Y%m%d'))
        data = {'Name':[],
           'Time':[]}
        df = pd.DataFrame(data,columns = ['Name','Time'])
        df.to_csv(f"Attendance/{attendance_date}/attendance.csv", sep=',',index=False)
    else:
        pass
           
        
    return attendance_date    


    

           
def markAttendance(name,date):

    
    with open(f'Attendance/{date}/attendance.csv','r+') as f:
        myDataList = f.readlines()    
        nameList = []
        for line in myDataList:        # read throug each line 
            entry = line.split(',')
            nameList.append(entry[0])    # appends the name in the first column
            
        if name not in nameList:         # If name not present already in the list then puts the time 
            now = datetime.now()
            dtString = now.strftime('%H%M%S')
            f.writelines(f'\n{name},{dtString}')
    
    
                        
            
encodeListKnown = findEncodings(images)
print('Encodings Completed !!!')

cap = cv2.VideoCapture(0)

while True:
    _,frame = cap.read()
    frame1 = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    frameS = cv2.resize(frame1,(0,0),None,0.25,0.25)
    
    
    
    
    # First we need to find the location of the faces in the frame and then encode the faces
    
    faceCurFrame = face_recognition.face_locations(frameS)
    encodeCurFrame = face_recognition.face_encodings(frameS,faceCurFrame)
    
    
    # Find the matches
    # Iterate through all the faces that we have through the faces in the current frame
    
    for encodeface,faceloc in zip(encodeCurFrame,faceCurFrame):
        matches = face_recognition.compare_faces(encodeListKnown,encodeface)
        
        # Find the distance of eac images
        # Lowest distance is the best match
        
        faceDistance = face_recognition.face_distance(encodeListKnown,encodeface)
        
        matchIndex = np.argmin(faceDistance)
        
        if matches[matchIndex]:
            name = classNames[matchIndex].upper()
            y1,x2,y2,x1 = faceloc
            y1,x2,y2,x1 = y1*4,x2*4,y2*4,x1*4
            cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),2)
            cv2.rectangle(frame,(x1,y2-30),(x2,y2),(0,255,0),-1)
            cv2.putText(frame,name,(x1+6,y2-6),cv2.FONT_HERSHEY_PLAIN,1,(255,255,255),2)
            markAttendance(name,create_folder())    # call the attendance funtion to put attendance of the identified page
            
    cv2.imshow('WebCam',frame)
    
    if cv2.waitKey(1) == 27:
        break
        
cap.release()
cv2.destroyAllWindows()

Encodings Completed !!!
